# Redis database

# A bit of context : what is Redis ?

Redis, which stands for REmote DIctionary Server, is a BSD-licensed, in-memory data structure store released in 2009 by Salvatore Sanfilippo. 

One of the big differences between Redis and other NoSQL databases is the data structures that Redis provides. Instead of working with a table abstraction, Redis developers can leverage data structures like strings, hashes, lists, sets, and sorted sets using commands similar to the collection operations in most programming languages. 

Redis has replication capabilities, a server-side scripting language (Lua), transactions, and different modes of disk persistence.

ex : système pour gérer els incrémentation lors de connexion sur srveeur/page web simultanée etc.. gère bien ça

gere automatiquement l'expiration des informations7

système de persistance :
- RDB : on crée un dump file en .rdb qui est sauvegardé et qui peux etre rechargé 
- AOF : ??? a voir


## One of the reasons that Redis is so fast in both read and write operations is that the database is held in memory (RAM) on the server

Expliquer histoire de redis les points clés et différences par rapport aux autres sql :
   - cache et session bonne gestion
   - systeme cle-valeur avancé incr, decincr etc.. pas mal puissant
    

## Brief explanation on redis commands : [nice tutorial](https://try.redis.io/?_ga=2.15710581.2114473602.1604997184-1839395559.1604575298)


## First, we have to install the necessary libraries for analyses :

In [132]:
# install the necessary modules :
pip install redis

Note: you may need to restart the kernel to use updated packages.


## Then, need to import the necessary libraries :

In [144]:
#import the redis client to communicate with
import redis
import random
import pprint
import json


## Connection
As previous databases, we need to open a connection with the Redis server to communicate with and execute code in a python script.

In [145]:
# Define our connection information for Redis

redis_host = "127.0.0.1"
redis_port = 6379


In [146]:
# Create connection object with :
# - the host, i.e the local name where the redis server is running
# - the port of redis server to communicate, default = 6379
# - the database number we want to work with, here we use the first one

r = redis.StrictRedis(host=redis_host, port=redis_port, db = 0)

## Now we're all set to start the analyse of the project !

## This is an hello world example!

In [147]:
def hello_redis():
    """Example Hello Redis Program"""
   
    # step 3: create the Redis Connection object
    # try:
   
    # The decode_repsonses flag here directs the client to convert the responses from Redis into Python strings
    # using the default encoding utf-8.  This is client specific.
    r = redis.Redis(host=redis_host, port=redis_port, decode_responses=True)
    
    # step 4: Set the hello message in Redis
    r.set("msg:hello", "Hello Redis!!!")
    r.set("msg:goodbye","Goodbye redis u suck son of a bitch")

    # step 5: Retrieve the hello message from Redis
    msg =[]
    # msg = r.get("msg:hello")
    msg = r.get("msg:goodbye")
    
    print(msg)

if __name__ == '__main__':
    hello_redis()

Goodbye redis u suck son of a bitch


## Few requests to understand Redis grammar in python

# Use case to understand Redis power : selling hats on an online website

It’s day one for the site, and we’re going to be selling three limited-edition hats. Each hat gets held in a Redis hash of field-value pairs, and the hash has a key that is a prefixed random integer , such as hat:56854717. Using the hat: prefix is Redis convention for creating a sort of namespace within a Redis database:

We create a small datasets with 3 differents hats

In [199]:

random.seed(444)
hats = {f"hat:{random.getrandbits(8)}": i for i in (
    {
        "color": "black",
        "price": 49.99,
        "style": "fitted",
        "quantity": 1000,
        "npurchased": 0,
    },
    {
        "color": "maroon",
        "price": 59.99,
        "style": "hipster",
        "quantity": 500,
        "npurchased": 0,
    },
    {
        "color": "green",
        "price": 99.99,
        "style": "baseball",
        "quantity": 200,
        "npurchased": 0,
    })
}
pprint.pprint(hats)

{'hat:3': {'color': 'green',
           'npurchased': 0,
           'price': 99.99,
           'quantity': 200,
           'style': 'baseball'},
 'hat:73': {'color': 'maroon',
            'npurchased': 0,
            'price': 59.99,
            'quantity': 500,
            'style': 'hipster'},
 'hat:79': {'color': 'black',
            'npurchased': 0,
            'price': 49.99,
            'quantity': 1000,
            'style': 'fitted'}}


### Now, we need to add them in the redis database thanks to server pipeline execution :

In [149]:
hats.items()

dict_items([('hat:79', {'color': 'black', 'price': 49.99, 'style': 'fitted', 'quantity': 1000, 'npurchased': 0}), ('hat:73', {'color': 'maroon', 'price': 59.99, 'style': 'hipster', 'quantity': 500, 'npurchased': 0}), ('hat:3', {'color': 'green', 'price': 99.99, 'style': 'baseball', 'quantity': 200, 'npurchased': 0})])

In [204]:
with r.pipeline() as pipe:
    for h_id, hat in hats.items():
        pipe.hmset(h_id, hat)
    pipe.execute()
    
r.bgsave()

<ipython-input-204-2a1a06282495>:3: DeprecationWarning: Pipeline.hmset() is deprecated. Use Pipeline.hset() instead.
  pipe.hmset(h_id, hat)


True

On regarde ce qu'il y a dans notre premier chapeau de notre objet hats

In [206]:
print(r.hgetall("hat:3"))

{b'color': b'green', b'price': b'99.99', b'style': b'baseball', b'quantity': b'200', b'npurchased': b'0'}


on regarde ce qu'il se passe si on modifie l'incrémentation des chapeaux comme si qqn achete un chapeau :
- la quantité diminue
- le nombre d'achat augmente 

In [156]:
r.hincrby("hat:3", "quantity", -1)
r.hincrby("hat:3", "npurchased", 1)

2

In [157]:
r.hget("hat:3", "npurchased")

b'2'

It isn’t really that simple, though. Changing the quantity and npurchased in two lines of code hides the reality that a click, purchase, and payment entails more than this. We need to do a few more checks to make sure we don’t leave someone with a lighter wallet and no hat:
 - Step 1: Check if the item is in stock, or otherwise raise an exception on the backend.
 - Step 2: If it is in stock, then execute the transaction, decrease the quantity field, and increase the npurchased field.
 - Step 3: Be alert for any changes that alter the inventory in between the first two steps (a race condition).


In [158]:
import logging

import redis

logging.basicConfig()


class OutOfStockError(Exception):

    """Raised when PyHats.com is all out of today's hottest hat"""


def buyitem(r: redis.Redis, itemid: int) -> None:

    with r.pipeline() as pipe:

        error_count = 0

        while True:

            try:

                # Get available inventory, watching for changes

                # related to this itemid before the transaction

                pipe.watch(itemid)

                nleft = r.hget(itemid, "quantity")

                if nleft > b"0":

                    pipe.multi()

                    pipe.hincrby(itemid, "quantity", -1)

                    pipe.hincrby(itemid, "npurchased", 1)

                    pipe.execute()

                    break

                else:

                    # Stop watching the itemid and raise to break out

                    pipe.unwatch()

                    raise OutOfStockError(

                        f"Sorry, {itemid} is out of stock!"

                    )

            except redis.WatchError:

                # Log total num. of errors by this user to buy this item,

                # then try the same process again of WATCH/HGET/MULTI/EXEC

                error_count += 1

                logging.warning(

                    "WatchError #%d: %s; retrying",

                    error_count, itemid

                )

    return None

 on a créé la fonction pour acheter un item et surveiller le stock qu'il reste, cela se fait assez facilement avec redis avec les incr etc... <br>
 
*a voir pour passer avec la logique big data derriere et creer vla les cleees etc...*

In [170]:
buyitem(r,"hat:79")
r.hmget("hat:79","quantity","npurchased")

[b'998', b'2']

maintenant on essaie d'en acheter plusieurs d'un coup et à terme tester le msg d'erreur qu'on a eu

In [171]:
for _ in range(56):
    buyitem(r,"hat:79")

In [172]:
r.hmget("hat:79", "quantity", "npurchased")

[b'942', b'58']

In [131]:
for _ in range(198):
    buyitem(r,"hat:79")    

OutOfStockError: Sorry, hat:79 is out of stock!

il faut remettre du stock car l'utilisateur à voulu acheter mais n'a pas pu !

### !!!!!!!!!éventuellement inclure : 
 - rajouter des options de monitoring du stock des objets et complexifier un peu l'offre donc voir modifier les chsoes au début pour faire genre de la gestion de maxi database de vetements 
 - une visu type barchat du stock pour chaque produit avec les requetes sur la db? à voir comment faire je maitrise pas trop ça
 - des visus complémentaires informant sur l'état des stocks, évolution des achats etc... à voir 

on s'intéresse à l'expiration et à comment on pourrait l'utiliser en pratique

In [176]:
from datetime import timedelta
r.setex("runner",timedelta(minutes=1.5), value =" careful! ")
r.ttl("runner")

90

In [196]:
r.ttl("runner")

78

A few days after its debut, PyHats.com has attracted so much hype that some enterprising users are creating bots to buy hundreds of items within seconds, which you’ve decided isn’t good for the long-term health of your hat business.

Now that you’ve seen how to expire keys, let’s put it to use on the backend of PyHats.com.

We’re going to create a new Redis client that acts as a consumer (or watcher) and processes a stream of incoming IP addresses, which in turn may come from multiple HTTPS connections to the website’s server.

The watcher’s goal is to monitor a stream of IP addresses from multiple sources, keeping an eye out for a flood of requests from a single address within a suspiciously short amount of time.

Some middleware on the website server pushes all incoming IP addresses into a Redis list with .lpush(). Here’s a crude way of mimicking some incoming IPs, using a fresh Redis database:

#### on verra is on met ça en place car il faut ouvrir une deuxième session et voir l'utilité du bloc d'adresse ip à voir..

## on s'intéresse maintenant à l'ajout d'objet *JSON-like* et voir comment cela est supporté dans redis

In [197]:
restaurant_484272 = {
    "name": "Ravagh",
    "type": "Persian",
    "address": {
        "street": {
            "line1": "11 E 30th St",
            "line2": "APT 1",
        },
        "city": "New York",
        "state": "NY",
        "zip": 10016,
    }
}
pprint.pprint(restaurant_484272)

{'address': {'city': 'New York',
             'state': 'NY',
             'street': {'line1': '11 E 30th St', 'line2': 'APT 1'},
             'zip': 10016},
 'name': 'Ravagh',
 'type': 'Persian'}


In [227]:
# ca ne va pas marcher car restaurant est un objet a part entiere 
r.hmset(484272, restaurant_484272)

<ipython-input-227-a6fd59feb95a>:2: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r.hmset(484272, restaurant_484272)


DataError: Invalid input of type: 'dict'. Convert to a bytes, string, int or float first.

In [229]:
# pour cela on utilise le fait de : Serialize the values into a string with something like json.dumps()
import json
r.set(484272, json.dumps(restaurant_484272))
r.get("484272")

b'{"name": "Ravagh", "type": "Persian", "address": {"street": {"line1": "11 E 30th St", "line2": "APT 1"}, "city": "New York", "state": "NY", "zip": 10016}}'

In [233]:
# il faut ensuite faire les commandes suivantes pour réobtenir l'objet correctement :
pprint.pprint(json.loads(r.get(484272)))

{'address': {'city': 'New York',
             'state': 'NY',
             'street': {'line1': '11 E 30th St', 'line2': 'APT 1'},
             'zip': 10016},
 'name': 'Ravagh',
 'type': 'Persian'}


No matter what serialization protocol you choose to go with, the concept is the same: you’re taking an object that is unique to Python and converting it to a bytestring that is recognized and exchangeable across multiple languages.

à voir ce que l'on fait ensuite, juste garder notre fonction buytime et améliorer les données peut-être?
flemme de faire la comparaison et de se replonger dans un truc bourbier encore mais on pourrait regarder les outputs de notre database existante et faire quelques graphs et montrer que c'est très rapide car tout est gardé en cache et voir avec les calculs avec la commande `%%time` au sein de chaque cellule jupyter

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


9

## Comparison with an other DB : MongoDB

### Qualitative approach
We are looking for differences in :
- grammar
- ease of use and prise en main
- setting up process (manipulation of our first database)
- access of ressources


### Quantitative approach
We are looking for differences in :
- time computation of requests
- memore/cache use for same data
